In [14]:
import json
import os
import numpy as np
import pandas as pd
import pygmo as pg
from pygmo import hypervolume

fol = "ExpMOEAD_Neig"

In [15]:
parent_folder_path = '../EvolutionaryFramework/results/MOEA_D/' + fol

target_folders = [folder for folder in os.listdir(parent_folder_path) if folder.startswith('2023_08_19_20:5')]

In [16]:
folder_averages = {}

metaCriteriaBool = False


for folder in target_folders:
    folder_path = os.path.join(parent_folder_path, folder)
    files = os.listdir(folder_path)

    metaBool = False
    mutation = 0
    crossover = 0
    keyType = ""
    criteria = []
    result = 0
    count = 0 

    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(folder_path, file)
            with open(file_path) as f:
                parsed_json = json.load(f)

        if metaBool == False:
            n = parsed_json["metaParameters"]["numberOfNeighbours"]
            metaBool = True

        if metaCriteriaBool == False:
            for criterion in parsed_json["metaParameters"]['criteria']:
                criteria.append(criterion)
            keyType = parsed_json["metaParameters"]["melodyKeyType"]
            

        data = []
        count += 1

        for elem in parsed_json:
            if elem == "experiment":
                for individual in parsed_json[elem]["generation_150"]:
                    dictionary = {}
                    for criterion in criteria:
                        dictionary[criterion] = -individual['fitness'][criterion]
                    data.append(dictionary)    
        data_df = pd.DataFrame(data)
        data_df = data_df.dropna()
    
        hyper = pg.hypervolume(data_df.to_numpy())
        result += hyper.compute([0.0, 0.0]) 

    folder_averages[keyType + " " + "generation_150 population_250 " + "n_" + str(n) + " " + folder] = result/count
        


output_file = 'JSON/' + 'hyperVolume_Neig2.json'
with open(output_file, 'w') as f:
    json.dump(folder_averages, f, indent=4)

print('Folder averages saved successfully.')


Folder averages saved successfully.


In [17]:
data_df

,SIMPLE_AND_OBVIOUS,COMPLICATED_AND_ENIGMATIC
0,-0.604589,-0.677106
1,-0.747810,-0.650695
2,-0.674200,-0.674578
3,-0.753959,-0.646774
4,-0.787313,-0.623547
...,...,...
68,-0.908467,-0.539131
69,-0.911586,-0.529331
70,-0.912671,-0.529250
71,-0.910512,-0.533051
